<a href="https://colab.research.google.com/github/LoveGarden/pytorch_case/blob/main/op_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1 张量创建：从无到有构建基础单元

tesor是PyTorch的核心数据结构，类似NumPy数组，但支持GPU加速和自动微分。以下是最常用的创建方式：

| op  | 功能描述| 关键参数 |
|----------|----------|----------|
torch.tensor|	从列表/元组/NumPy数组转换为张量|	data(输入数据)、requires_grad(是否追梯度)
torch.zeros|	生成全0张量	|size(张量形状，如(2,3))
torch.ones|	生成全1张量	|size(张量形状)
torch.rand|	生成[0,1)均匀分布的随机张量	|size(张量形状)
torch.randn|	生成均值0、标准差1的正态分布随机张量	|size(张量形状)
torch.arange|	生成等差数列张量（类似Python的range）|	start(起始值)、end(终止值)、step(步长)
torch.linspace|	生成固定数量的均匀间隔张量（包含终点）	|start、end、steps(间隔数)
torch.logspace|	生成对数尺度的均匀间隔张量	|start、end、steps、base(对数基，默认10)
torch.eye|	生成单位矩阵（对角线1，其余0）	|n(行数)、m(列数，默认等于n)
torch.empty|	生成未初始化张量（内存残留值，速度快）| size(张量形状)
torch.full|	生成全为指定值的张量	|size(形状)、fill_value(填充值）



In [1]:
import torch

# 1. 从列表创建张量，指定追梯度
a = torch.tensor([1, 2, 3], requires_grad=True)
print("torch.tensor创建：", a)  # 输出：tensor([1, 2, 3], requires_grad=True)

# 2. 生成2行3列的全0张量
b = torch.zeros(2, 3)
print("torch.zeros创建：\n", b)  # 输出2x3的0矩阵

# 3. 生成0到10、步长2的等差数列
c = torch.arange(0, 10, 2)
print("torch.arange创建：", c)  # 输出：tensor([0, 2, 4, 6, 8])

RuntimeError: Only Tensors of floating point and complex dtype can require gradients